<p>Dieses Nodebook dient dazu einfach einmal herauszufinden was die optimale Tiefe bei dem Datenset ist</p>

In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

C:\Users\janth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#dieser Teil des codes probiert ein Minimum in den Daten zu finden und nutzt dabei optuna was das automatisch macht

In [6]:
"""
from google.colab import files
import pandas as pd
import io

# Datei-Upload starten
uploaded = files.upload()

# Erste (oder einzige) Datei automatisch in DataFrame laden
for name, file in uploaded.items():
    df = pd.read_csv(io.BytesIO(file))
    break  # Nur die erste Datei verwenden
"""

Saving Modellierung_05.csv to Modellierung_05.csv


In [2]:
df = pd.read_csv("../Ergebnisse/Modellierungen/V05/Modellierung_05.csv")

In [3]:
df = df.set_index("timestamp")

In [4]:
df

,Deutschland (Positiv),Deutschland (Negativ),Solar,Wind Offshore,Wind Onshore,Residuallast,_sin_year,_cos_year,_sin_week,_cos_week,...,precipitation_Min,precipitation_Max,wind_speed_100m_Mean,wind_speed_100m_Var,wind_speed_100m_Min,wind_speed_100m_Max,wind_gusts_10m_Mean,wind_gusts_10m_Var,wind_gusts_10m_Min,wind_gusts_10m_Max
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-30 22:00:00+00:00,390.620,350.816,3.0,1317.0,13862.0,37257.0,0.025818,-0.999667,0.433884,-0.900969,...,0.0,7.000,22.300,114.306667,4.800,38.400,21.9900,139.374333,7.6,45.000
2022-06-30 22:15:00+00:00,162.736,263.656,3.0,1459.0,13702.0,37019.0,0.025818,-0.999667,0.433884,-0.900969,...,0.0,6.525,22.285,100.867389,5.925,38.450,21.9200,128.735806,8.3,44.550
2022-06-30 22:30:00+00:00,928.544,676.444,3.0,1480.0,13340.0,36835.0,0.025818,-0.999667,0.433884,-0.900969,...,0.0,6.050,22.270,89.557333,7.050,38.500,21.8500,119.708889,9.0,44.100
2022-06-30 22:45:00+00:00,1212.684,825.452,3.0,1428.0,12910.0,36779.0,0.025818,-0.999667,0.433884,-0.900969,...,0.0,5.575,22.255,80.376500,8.175,38.550,21.7800,112.293583,9.7,43.650
2022-06-30 23:00:00+00:00,1402.408,963.996,3.0,1352.0,12591.0,37140.0,0.025818,-0.999667,0.433884,-0.900969,...,0.0,5.100,22.240,73.324889,9.300,38.600,21.7100,106.489889,10.4,43.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 12:30:00+00:00,1178.284,1221.068,10372.0,2787.0,37618.0,15983.0,0.017213,0.999852,0.974928,-0.222521,...,0.0,0.000,58.150,497.207778,21.800,94.650,68.2950,507.489694,23.2,98.800
2025-01-01 12:45:00+00:00,1184.488,1234.224,9531.0,3055.0,37999.0,16029.0,0.017213,0.999852,0.974928,-0.222521,...,0.0,0.000,58.115,491.693500,21.050,94.025,68.4325,507.765701,23.3,98.000
2025-01-01 13:00:00+00:00,1257.392,1004.504,8525.0,3003.0,37911.0,15798.0,0.017213,0.999852,0.974928,-0.222521,...,0.0,0.000,58.080,487.850667,20.300,93.400,68.5700,511.042333,23.4,97.200


In [5]:
y_pos = df["Deutschland (Positiv)"]
y_neg = df["Deutschland (Negativ)"]
X = df.drop(["Deutschland (Positiv)", "Deutschland (Negativ)"], axis=1)

In [6]:
y = y_pos

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_small, _, y_small, _ = train_test_split(X, y, train_size=0.1, random_state=42)

In [8]:
df.columns

Index(['Deutschland (Positiv)', 'Deutschland (Negativ)', 'Solar',
       'Wind Offshore', 'Wind Onshore', 'Residuallast', '_sin_year',
       '_cos_year', '_sin_week', '_cos_week', '_sin_day', '_cos_day',
       'load forecast error Deutschland', 'gradient Deutschland',
       'Feiertagsindex', 'Cross_Border_Summe physical flow',
       'Cross_Border_Summe difference flow', 'cloudcover_Mean',
       'cloudcover_Var', 'cloudcover_Min', 'cloudcover_Max',
       'temperature_2m_Mean', 'temperature_2m_Var', 'temperature_2m_Min',
       'temperature_2m_Max', 'relative_humidity_2m_Mean',
       'relative_humidity_2m_Var', 'relative_humidity_2m_Min',
       'relative_humidity_2m_Max', 'precipitation_Mean', 'precipitation_Var',
       'precipitation_Min', 'precipitation_Max', 'wind_speed_100m_Mean',
       'wind_speed_100m_Var', 'wind_speed_100m_Min', 'wind_speed_100m_Max',
       'wind_gusts_10m_Mean', 'wind_gusts_10m_Var', 'wind_gusts_10m_Min',
       'wind_gusts_10m_Max'],
      dtype='obje

In [ ]:
def objective(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int("n_estimators", 100, 800),
        max_depth=trial.suggest_int("max_depth", 2, 40),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        subsample=trial.suggest_float("subsample", 0.5, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.5, 1.0),
        random_state=42,
        n_jobs=-1
    )
    scores = cross_val_score(model, X_small, y_small, cv=5, scoring="neg_mean_squared_error")
    return -scores.mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Beste Parameter:", study.best_params)

[I 2025-06-30 20:36:25,439] A new study created in memory with name: no-name-4fe36ddc-ecd1-4c74-8b79-64c042bad007
